In [2]:
from IPython import display
from itertools import product
import os
import pytz
import sys
import time as tt
import warnings
os.environ['PROJ_LIB']=os.path.join(os.environ['HOME'],'.anaconda3','share','proj')

warnings.filterwarnings('ignore', category=RuntimeWarning)

from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset as nc, num2date
import pandas as pd
import pytz
sys.path.append('/home/unimelb.edu.au/mbergemann/.local/lib/python3.6/site-packages/')


from tint import Cell_tracks, animate, helpers
from TintTrack import cold_pool_grids, creat_tracks, get_mintime

In [3]:
F='/home/unimelb.edu.au/mbergemann/Data/Extremes/UM/darwin/RA1T/um-0p44km-11100000-vert_cent_20061112_0000-20061112_1200-2.5km.nc'
F='/home/unimelb.edu.au/mbergemann/Data/Extremes/UM/darwin/RA1T/um-0p44km-11100000-vert_cent_20061110_0000-20061119_0600-2.5km.nc'
R=F.replace('vert_cent','rain')
masF = '/home/unimelb.edu.au/mbergemann/Data/Extremes/UM/darwin/RA1T/mask_um2p5.nc'
with nc(F) as f:
    time = num2date(f.variables['t'][:], f.variables['t'].units)
    lon = f.variables['lon'][:]
    lat = f.variables['lat'][:]
    start, end = time[0], time[-1]
    slices = helpers.get_times(f.variables['t'], start, end)
timezone = pytz.timezone('Australia/Darwin')
utc = pytz.UTC
m = Basemap(llcrnrlat=min(lat), llcrnrlon=min(lon), urcrnrlat=max(lat), urcrnrlon=max(lon),
                            resolution='f', area_thresh=0.5)

In [4]:
%matplotlib inline
from TintTrack import cold_pool_grids, creat_tracks
first = True
fig=plt.figure(figsize=(9,4))
ax = fig.add_subplot(111)

first = True
vmin, vmax = 0, 0
cmap = cm.GMT_haxby
cmap.set_under('w')
cmap.set_bad('w')
while True:
    try:    
        for s in slices:
            gr = (i for i in cold_pool_grids(nc(F), nc(R), s, lon, lat, maskfile=masF))
            for data in gr:
                ts = pd.Timestamp(data['time']).tz_localize(utc).tz_convert(timezone).tz_localize(None)
                #print(data['data'][0].mean(), data['data'][0].min(), data['data'][0].max(), data['data'][0].std())
                #break
                if first:
                    im = m.pcolormesh(lon, lat, -data['data'][0], vmax=-0.03, vmin=-3, cmap=cmap, shading='gouraud')
                    cbar = m.colorbar(im, location='bottom')
                    cbar.set_label('$\\theta_p$ field pertubation', fontsize=14)
                    m.drawcoastlines(linewidth=0.3)
                    first = False
                else:
                    im.set_array(-data['data'][0].ravel())
                ax.set_title(ts.strftime('%_d/%_m/%Y %H:%M LT'), fontsize=16)
                fig.canvas.draw()
                display.clear_output(wait=True)
                display.display(plt.gcf())
                tt.sleep(0.2)
                
    except KeyboardInterrupt:
        da = ax.cla(), fig.clf(), plt.close()
        break
        
d = display.clear_output(wait=True)
d= display.display(plt.gcf())

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [8]:
#The cold pool function
def create_coldpooltracks(rainF, vertF, trackdir, start=None, end=None, overwrite=True, animate_movie=True, timename='t',
                          lonname='lon', latname='lat', vmin=0.0, vmax=5, maskfile=None, **kwargs):
    '''
    Create tintV2 tracks 

    Arguments : 
        dataF (str) : the netCDF file that contains the rainfall data
    Key word args:
        start : If start is given first time step of the tracking, if None 
                start is the first time step of the data.
                Format should be 'YYYY-mm-dd hh:MM' (default : None)
        end  : If send is given last time step of the tracking, if None 
                end is the last time step of the data (
                Format should be 'YYYY-mm-dd hh:MM' default : None)
        overwrite : If a track already exist, delete it an crate a new one,
                    (dfault : True)
        animate  : Create a movie of the tracked systems
        varname : Name of the rain variable
        timename : Name of the time variable
        lonname : Name of the longitude variable
        latname : Name of the latitude variable
        group : Subgroup where the data is stored
    '''

    trackdir = os.path.join(trackdir, 'Tracking')
    sys.stdout.flush()
    sys.stdout.write('\r Creating tracks for %s\n' % os.path.basename(vertF))
    sys.stdout.flush()
    try:
        os.makedirs(os.path.join(trackdir, 'video'))
    except:
        pass

    try:
        out_name = kwargs['out_name']
        del kwargs['out_name']
    except KeyError:
        out_name = 'coldpool_tracks'

    try:
        ggrids = kwargs['ggrids']
        del kwargs['ggrids']
    except KeyError:
        ggrids = cold_pool_grids
    with nc(vertF) as ncg:
        g = ncg
        with nc(rainF) as ncf:
            if type(start) == type('a') and type(end) == type('a'):
                slices = helpers.set_times(start, end, g.variables[timename])
            else:
                try:
                    slices = helpers.spl(g.variables['ispresent'][:],g.variables[timename][:])
                except KeyError:
                    start = num2date(g.variables[timename][:],
                                     g.variables[timename].units)[0]
                    end = num2date(g.variables[timename][:],
                                   g.variables[timename].units)[-1]
                    slices = helpers.get_times(g.variables[timename], start, end)

            lats = ncf.variables[latname][:]
            lons = ncf.variables[lonname][:]
            if len(lats.shape) == 2:
                lats = lats[:, 0]
            if len(lons.shape) == 2:
                lons = lons[0, :]

            x = lons[len(lons)//2]
            y = lats[len(lats)//2]
            grids = []
            for s in slices:
                ani = False
                gr = (i for i in ggrids(g, ncf, s, lons, lats,
                                       tn=timename, maskfile=maskfile, **kwargs))
                anim = (i for i in ggrids(g, ncf, s, lons, lats, tn=timename,
                                          maskfile=maskfile,**kwargs))
                start = num2date(g.variables[timename][s[0]],
                             g.variables[timename].units)

                end = num2date(g.variables[timename][s[-1]],
                               g.variables[timename].units)
                suffix = '%s-%s' % (start.strftime('%Y_%m_%d_%H'),
                                end.strftime('%Y_%m_%d_%H'))
                tracks_obj = Cell_tracks()
                tracks_obj.params['MIN_SIZE'] = 2
                tracks_obj.params['FIELD_THRESH'] = vmin
                track_file = os.path.join(trackdir, 'coldpool_tracks_%s.hdf5' % (suffix))
                if not os.path.isfile(track_file) or overwrite:
                    ncells = tracks_obj.get_tracks(gr, (x, y))
                    if ncells > 2:
                        tracks_obj.tracks.to_hdf(track_file, 'coldpool', mode='a',
                                              format='table')
                        ani = True
                    else:
                        ani = False
            

                if ani and animate_movie:
                    animate(tracks_obj, anim,
                           os.path.join(trackdir, 'video',
                                        '%s_%s.mp4' % (out_name, suffix)),
                            overwrite=overwrite, dt=9.5, vmin=vmin, vmax=vmax,**kwargs)

In [4]:
# Define our ensemble member
ensembles = ('20061109T1200Z', '20061109T1800Z', '20061110T0000Z',
             '20061110T0600Z', '20061110T1200Z', '20061110T1800Z',
             '20061111T0000Z', '20061111T1200Z')
UMdir = os.path.join(os.getenv('HOME'), 'Data', 'Extremes', 'UM', 'darwin', 'RA1T')
maskfile = os.path.join(UMdir, 'mask_um2p5.nc')
remap_res = '2.5km'
animate_movie = True
Simend = '20061119_0600-%s' % remap_res

In [7]:
umrainf, start, end = get_mintime(ensembles, Simend, UMdir)
umvert, start, end = get_mintime(ensembles, Simend, UMdir, vn='vert_cent')
umfiles = zip(umrainf, umvert)
ensn = 0
for rainfiles, vertfiles in umfiles:
    for n, res in enumerate(('1.33km', '0.44km')):
        umrainf, umvertf = rainfiles[n], vertfiles[n]
        res, ens = os.path.basename(umrainf).split('-')[1:3]
        outDir = os.path.join(UMdir, ensembles[ensn], 'darwin', res)
        create_coldpooltracks(umrainf, umvertf, outDir, maskfile=maskfile, start=start, end=end,
                              keep_frames=True, animate_movie=False)
    ensn += 1

 Creating tracks for um-1p33km-11091200-vert_cent_20061109_1200-20061119_0600-2.5km.nc
No cells found in scan 1439     

time elapsed 1.4 minutes
 Creating tracks for um-0p44km-11091200-vert_cent_20061109_1200-20061119_0600-2.5km.nc
No cells found in scan 1439     

time elapsed 1.6 minutes
 Creating tracks for um-1p33km-11091800-vert_cent_20061109_1800-20061119_0600-2.5km.nc
Writing tracks for scan 1403    

time elapsed 1.6 minutes
 Creating tracks for um-0p44km-11091800-vert_cent_20061109_1800-20061119_0600-2.5km.nc
Writing tracks for scan 1403    

time elapsed 1.5 minutes
 Creating tracks for um-1p33km-11100000-vert_cent_20061110_0000-20061119_0600-2.5km.nc
Writing tracks for scan 1367    

time elapsed 1.6 minutes
 Creating tracks for um-0p44km-11100000-vert_cent_20061110_0000-20061119_0600-2.5km.nc
Writing tracks for scan 1367    

time elapsed 1.6 minutes
 Creating tracks for um-1p33km-11100600-vert_cent_20061110_0600-20061119_0600-2.5km.nc
Writing tracks for scan 1331    

tim